In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Set the path to the dataset directory
dataset_dir = r"C:/Users/Lenovo/Downloads/archive (7)/PlantDiseasesDataset"

# Set the desired image dimensions
image_height, image_width = 224, 224

# Set the batch size and number of epochs
batch_size = 32
epochs = 10

# Set the number of classes
num_classes = [4, 4, 3, 2]

# Function to generate the path to each category's train and validation folders
def get_category_path(category):
    return os.path.join(dataset_dir, category, "train")

# Load the MobileNetV2 model without the top classification layer
base_model = MobileNetV2(
    include_top=False,
    input_shape=(image_height, image_width, 3),
    weights='imagenet'
)

# Add a global average pooling layer and a dense output layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Loop through each category and train a separate model
for i, category in enumerate(os.listdir(dataset_dir)):
    category_path = get_category_path(category)

    if not os.path.isdir(category_path):
        continue

    train_generator = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    ).flow_from_directory(
        category_path,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = ImageDataGenerator(
        rescale=1.0 / 255,
        validation_split=0.2
    ).flow_from_directory(
        category_path,
        target_size=(image_height, image_width),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation'
    )

    output = Dense(num_classes[i], activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size
    )

    # Evaluate the model on the validation set
    score = model.evaluate(validation_generator, verbose=0)
    print(f"Category: {category}")
    print("Validation loss:", score[0])
    print("Validation accuracy:", score[1])

    # Save the trained model for each category
    model.save(f"{category}_plant_disease_detection_model.h5")


Found 10501 images belonging to 4 classes.
Found 2623 images belonging to 4 classes.
Epoch 1/10
328/328 [==============================] - 3391s 10s/step - loss: 0.1533 - accuracy: 0.9528 - val_loss: 5.3804 - val_accuracy: 0.6817
Epoch 2/10
328/328 [==============================] - 3213s 10s/step - loss: 0.0610 - accuracy: 0.9800 - val_loss: 9.6822 - val_accuracy: 0.6177
Epoch 3/10
328/328 [==============================] - 4533s 12s/step - loss: 0.0715 - accuracy: 0.9764 - val_loss: 10.5193 - val_accuracy: 0.5448
Epoch 4/10
328/328 [==============================] - 3525s 11s/step - loss: 0.0773 - accuracy: 0.9746 - val_loss: 4.2306 - val_accuracy: 0.7932
Epoch 5/10
328/328 [==============================] - 4110s 13s/step - loss: 0.0407 - accuracy: 0.9858 - val_loss: 3.5972 - val_accuracy: 0.6462
Epoch 6/10
328/328 [==============================] - 3279s 10s/step - loss: 0.0882 - accuracy: 0.9721 - val_loss: 5.8597 - val_accuracy: 0.7404
Epoch 7/10
328/328 [========================